
# 1 Construir un cluster en MongoDB y conectarse.


In [ ]:
!pip install pymongo[srv]

In [ ]:
# Import Libraries

import pymongo                            # Library to access MongoDB
from pymongo import MongoClient           # Imports MongoClient
import pandas as pd                       # Library to work with dataframes
import folium                             # Library to visualize a map

In [ ]:

uri = "mongodb+srv://somethingtoconnecttothecluster"

# Create a new client and connect to the server
client = MongoClient(uri)

print(client)

MongoClient(host=['ac-pjvqtei-shard-00-02.cjkz7i5.mongodb.net:27017', 'ac-pjvqtei-shard-00-01.cjkz7i5.mongodb.net:27017', 'ac-pjvqtei-shard-00-00.cjkz7i5.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-qwsl4i-shard-0', tls=True)


In [ ]:
db = client.sample_restaurants
       # Set the database to work on
db.list_collection_names()             # List the collections available


['neighborhoods', 'restaurants']

In [ ]:
collection = db.restaurants

In [ ]:
num_documents = collection.count_documents({'_id' : {'$exists' : 1}})
print ( 'Number of documents in database = ' + str(num_documents) )
list ( collection.find().limit(1) )

Number of documents in database = 25359


[{'_id': ObjectId('5eb3d668b31de5d588f4292a'),
  'address': {'building': '2780',
   'coord': [-73.98241999999999, 40.579505],
   'street': 'Stillwell Avenue',
   'zipcode': '11224'},
  'borough': 'Brooklyn',
  'cuisine': 'American',
  'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
    'grade': 'A',
    'score': 5},
   {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
   {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
   {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
  'name': 'Riviera Caterer',
  'restaurant_id': '40356018'}]

In [ ]:
# Recuperar un documento de la colección y mostrar su estructura
documento = collection.find_one()
list ( documento )

['_id', 'address', 'borough', 'cuisine', 'grades', 'name', 'restaurant_id']

In [ ]:
print("\nTipos de datos de cada variable:")
print("")
for key, value in documento.items():
    print(f"{key}: {type(value)}")


Tipos de datos de cada variable:

_id: <class 'bson.objectid.ObjectId'>
address: <class 'dict'>
borough: <class 'str'>
cuisine: <class 'str'>
grades: <class 'list'>
name: <class 'str'>
restaurant_id: <class 'str'>


# Queries

Tiene sentido con los datos del ejercicio hacer funcionalidades para un búscador de restaurantes. Voy a intentar reproducir algunas queries o filtros que podría querer un usuario en un buscador (Tenedor, Trivago o Maps).

## Buscar por Cocina

In [ ]:
# Filtrar por cocina
def filtrar_cocina(my_input):
  cuisine = str(my_input).strip()

  result_cocina = collection.aggregate([
      {'$match' : {'cuisine' : cuisine}}
  ])
  return result_cocina

In [ ]:
collection.distinct('cuisine')

['Afghan',
 'African',
 'American',
 'Armenian',
 'Asian',
 'Australian',
 'Bagels/Pretzels',
 'Bakery',
 'Bangladeshi',
 'Barbecue',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Brazilian',
 'CafÃ©/Coffee/Tea',
 'Café/Coffee/Tea',
 'Cajun',
 'Californian',
 'Caribbean',
 'Chicken',
 'Chilean',
 'Chinese',
 'Chinese/Cuban',
 'Chinese/Japanese',
 'Continental',
 'Creole',
 'Creole/Cajun',
 'Czech',
 'Delicatessen',
 'Donuts',
 'Eastern European',
 'Egyptian',
 'English',
 'Ethiopian',
 'Filipino',
 'French',
 'Fruits/Vegetables',
 'German',
 'Greek',
 'Hamburgers',
 'Hawaiian',
 'Hotdogs',
 'Hotdogs/Pretzels',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Indian',
 'Indonesian',
 'Iranian',
 'Irish',
 'Italian',
 'Japanese',
 'Jewish/Kosher',
 'Juice, Smoothies, Fruit Salads',
 'Korean',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Moroccan',
 'Not Listed/Not Applicable',
 'Nuts/Confectionary',
 'Other',

In [ ]:
# Test sencillo de filtrar_cocina

test_result = filtrar_cocina('Spanish')

df_result = pd.DataFrame(test_result)
num_documents_test = df_result.shape[0]

print ( 'Number of documents in the result = ' + str(num_documents_test) + ' vs number of docs in the original database => ' + str(num_documents))

df_result.iloc[0]

Number of documents in the result = 637 vs number of docs in the original database => 25359


_id                                       5eb3d668b31de5d588f4299b
address          {'building': '113', 'coord': [-73.9979214, 40....
borough                                                  Manhattan
cuisine                                                    Spanish
grades           [{'date': 2014-07-25 00:00:00, 'grade': 'A', '...
name                                        Spain Restaurant & Bar
restaurant_id                                             40365472
Name: 0, dtype: object


<br>

## Buscar por Valoración
Primero vemos los posibles valores del dataset

In [ ]:
unique_grades = set()
unique_scores = set()

for item in collection.find():
    grades = [grade["grade"] for grade in item.get("grades", [])]
    scores = [grade["score"] for grade in item.get("grades", [])]

    unique_grades |= set(grades)
    unique_scores |= set(scores)

print("Unique Grades:", unique_grades)
print("Unique Scores:", unique_scores)

Unique Grades: {'Not Yet Graded', 'P', 'Z', 'B', 'C', 'A'}
Unique Scores: {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, None, 70, 71, 69, 73, 75, 76, 77, 78, 79, 80, 81, 82, 84, 86, 89, 90, 92, 98, 131, -1}


Aquí parece que las letras están puestas un poco al azar y no se entiendo que significado tiene la 'Z' o la 'P'. Para aclararlo vamos a hacer una query que nos devuelva el valor medio, el mínimo y el máximo para cada letra.

In [ ]:
for g in unique_grades:

  result = collection.aggregate([
        {'$unwind' : '$grades'},
        {'$project' : {'_id' : 1, 'grade' : '$grades.grade', 'score' : '$grades.score'}},
        {'$match' : {'grade' : g}},
        {'$group' : {'_id' : '$grade', 'avg' : {'$avg' : '$score'}, 'min' : {'$min' : '$score'}, 'max' : {'$max' : '$score'}}},
        {'$project' : {'_id' : 0, 'grade' : '$_id', 'avg' : '$avg', 'min' : '$min', 'max' : '$max'}}
    ])

  df = pd.DataFrame(result)
  print(df)

            grade        avg  min  max
0  Not Yet Graded  15.853516    0   75
  grade       avg  min  max
0     P  5.506266    0   43
  grade        avg  min  max
0     Z  23.702319   -1   71
  grade       avg  min  max
0     B  20.00246   -1   57
  grade        avg  min  max
0     C  30.700795   -1  131
  grade       avg  min  max
0     A  9.021954    0   27


Vamos a filtrar los valores de score -1

In [ ]:
for g in unique_grades:

  result = collection.aggregate([
        {'$unwind' : '$grades'},
        {'$project' : {'_id' : 1, 'grade' : '$grades.grade', 'score' : '$grades.score'}},
        {'$match' : {'grade' : g, 'score' :{'$ne' : -1}}},
        {'$group' : {'_id' : '$grade', 'avg' : {'$avg' : '$score'}, 'min' : {'$min' : '$score'}, 'max' : {'$max' : '$score'}}},
        {'$project' : {'_id' : 0, 'grade' : '$_id', 'avg' : '$avg', 'min' : '$min', 'max' : '$max'}}
    ])

  df = pd.DataFrame(result)
  print(df)

            grade        avg  min  max
0  Not Yet Graded  15.853516    0   75
  grade       avg  min  max
0     P  5.506266    0   43
  grade        avg  min  max
0     Z  23.739326    0   71
  grade        avg  min  max
0     B  20.017469    0   57
  grade        avg  min  max
0     C  30.720967    0  131
  grade       avg  min  max
0     A  9.021954    0   27


Es evidente que es mejor guiarse por los Score y no por las letras de la variable Grades. Aunque, podemos ver por la media de los grupos que las letras A y P contienen los grupos más bajos y el C es el grupo con valor más altos de media.

 Otra cosa a tener en cuenta es que hay scores mayores de 100 que no ayudan a una valoración coherente de los datos. Habría que normalizar los datos para que fueran más facilmente visualizables y comparables. Esto lo dejo como anotación y trabajo recomendado para la implementación del buscador, puesto que, aquí solo me centraré en la parte relacionada con el motor de búsqueda.

In [ ]:
result_valoracion = collection.aggregate([
      {'$unwind' : '$grades'},
      {'$group' : {'_id' : '$name', 'avg_score' : {'$avg' : '$grades.score'}}},
      {'$sort' : {'avg_score' : -1}},
      {'$limit' : 10}
  ])

df_result = pd.DataFrame(result_valoracion)
print(df_result)
df_result.shape

                         _id  avg_score
0        Juice It Health Bar       75.0
1      Golden Dragon Cuisine       73.0
2    Chelsea'S Juice Factory       69.0
3        Palombo Pastry Shop       69.0
4                      Koyla       61.0
5         K & D Internet Inc       61.0
6           Ivory D O S  Inc       60.0
7        Ab Halal Restaurant       58.0
8                     Espace       56.0
9  Wing Hing Chinese Cuisine       56.0


(10, 2)

#### Ahora creamos una funcion para devolver los top n sitios por valoración media



In [ ]:
def top_restaurantes(my_input):
  n = int(my_input)

  result_restaurantes = collection.aggregate([
      {'$unwind' : '$grades'},
      {'$group' : {'_id' : '$name', 'avg_score' : {'$avg' : '$grades.score'}}},
      {'$sort' : {'avg_score' : -1}},
      {'$limit' : n}
  ])
  return result_restaurantes

In [ ]:
## Simple test
test_result = top_restaurantes(5)

df_result = pd.DataFrame(test_result)
df_result

,_id,avg_score
0,Juice It Health Bar,75.0
1,Golden Dragon Cuisine,73.0
2,Chelsea'S Juice Factory,69.0
3,Palombo Pastry Shop,69.0
4,Koyla,61.0


<br>

### Buscar por Municipio - Borough

In [ ]:
# Filtrar por Municipio
def filtrar_municipio(my_input):
  municipio = str(my_input).strip()

  result_municipio = collection.aggregate([
      {'$match' : {'borough' : municipio}}
  ])
  return result_municipio

In [ ]:
collection.distinct('borough')

['Bronx', 'Brooklyn', 'Manhattan', 'Missing', 'Queens', 'Staten Island']

In [ ]:
# Test sencillo de filtrar_municipio

test_result = filtrar_municipio('Queens')

df_result = pd.DataFrame(test_result)
num_documents_test = df_result.shape[0]

print ( 'Number of documents in the result = ' + str(num_documents_test) + ' vs number of docs in the original database => ' + str(num_documents))

df_result.iloc[0]

Number of documents in the result = 5656 vs number of docs in the original database => 25359


_id                                       5eb3d668b31de5d588f4292f
address          {'building': '97-22', 'coord': [-73.8601152, 4...
borough                                                     Queens
cuisine                                              Jewish/Kosher
grades           [{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...
name                                            Tov Kosher Kitchen
restaurant_id                                             40356068
Name: 0, dtype: object

# Creamos un buscador

Creamos un buscador que filtra por municipio y cocina. Luego devuelve los n restaurantes mejor valorados que cumplan esos filtros

In [ ]:
import random as rd

### Unimos todas las funciones en una

In [ ]:
def top_restaurant_byCocina_byMunicipio(m, c, n):
  #####
  # 1 -> filtra por Municipio (m)
  # 2 -> filtra por Conicna (c)
  # 3 -> Saca el las (n) opciones con más valoración media

  m = str(m).strip()
  c = str(c).strip()
  n = int(n)



  result_final = collection.aggregate([
      {'$match' : {'borough' : m}},
      {'$match' : {'cuisine' : c}},
      {'$unwind' : '$grades'},
      {'$group' : {'_id' : '$name', 'avg_score' : {'$avg' : '$grades.score'}}},
      {'$sort' : {'avg_score' : -1}},
      {'$limit' : n}
  ])

  return result_final


In [ ]:
municipios = collection.distinct('borough')
cocina = collection.distinct('cuisine')

In [ ]:
## Mejores 3 restaurantes

municipio_choice = rd.choice(municipios)
cocina_choice = rd.choice(cocina)
n = 3

print("My Borough choice is : " + municipio_choice)
print("My Cuisie choice is : " + cocina_choice)
print("Queremos el top " + str(n) + " de los restaurantes")

result_final = top_restaurant_byCocina_byMunicipio(municipio_choice, cocina_choice, n)

df_result = pd.DataFrame(result_final)
df_result

My Borough choice is : Bronx
My Cuisie choice is : Jewish/Kosher
Queremos el top 3 de los restaurantes


,_id,avg_score
0,Block Pizza,16.00
1,Gruenebaum'S Bakery &Cafe,15.75
2,Delicious On Lydig,12.80
